# Mobile App Data Preparation

### Step 1
**Import functions** and provide the **Google Cloud Platform credentials**.

In [54]:
# Import functions
import pandas as pd
import numpy as np
from google.cloud import storage
from google.cloud import bigquery
import pandas as pd
from io import BytesIO
import os

In [53]:
# Credentials to access my Google CLoud
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/pietr/Visual Studio/mobile_dataset/app-usage-414021-1197778b794d.json"

### Step 2
**Download** the **raw data** file from Google Cloud Storage and convert it to a **pandas dataframe**.

In [126]:
def load_csv_to_dataframe(bucket_name, blob_name, separator):
    # Create a storage client.
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    # Download the blob's contents and decode as a string.
    data = blob.download_as_text()

    # Use StringIO to convert the string data into a file-like object so pandas can read it.
    string_buffer = StringIO(data)

    # Read the data into a pandas DataFrame.
    df = pd.read_csv(string_buffer, delimiter = separator)

    return df

In [141]:
# Bucket and file name on Google Cloud
bucket_name = 'event-app-usage'
blob_name = 'app_data_raw.tsv'
separator = '\t'
df = load_csv_to_dataframe(bucket_name, blob_name, separator)

### Step 3
**Exploratory** data analysis.

In [80]:
# Display the first rows of the DataFrame
df.head()

,user_id,session_id,timestamp,app_name,event_type
0,0,1,2018-01-16 06:01:05,Minesweeper Classic (Mines),Opened
1,0,1,2018-01-16 06:01:05,Minesweeper Classic (Mines),Closed
2,0,1,2018-01-16 06:01:07,Minesweeper Classic (Mines),Opened
3,0,1,2018-01-16 06:01:07,Minesweeper Classic (Mines),Closed
4,0,1,2018-01-16 06:01:08,Minesweeper Classic (Mines),Opened


In [81]:
# Display the last rows of the DataFrame
df.tail()

,user_id,session_id,timestamp,app_name,event_type
3658584,291,76247,2018-04-06 14:35:15,Facebook,Closed
3658585,291,76247,2018-04-06 14:35:15,Facebook,Opened
3658586,291,76247,2018-04-06 14:35:37,Facebook,Closed
3658587,291,76247,2018-04-06 14:35:37,Facebook,Opened
3658588,291,76247,2018-04-06 14:35:52,Facebook Messenger,User Interaction


In [82]:
# Summary of data types, missing values, and data shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3658589 entries, 0 to 3658588
Data columns (total 5 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   user_id     int64 
 1   session_id  int64 
 2   timestamp   object
 3   app_name    object
 4   event_type  object
dtypes: int64(2), object(3)
memory usage: 139.6+ MB


In [83]:
# Summary statistics for numerical columns
df.describe()

,user_id,session_id
count,3.658589e+06,3.658589e+06
mean,1.466050e+02,3.501439e+04
std,8.504212e+01,2.215092e+04
min,0.000000e+00,1.000000e+00
25%,7.100000e+01,1.540500e+04
50%,1.380000e+02,3.128200e+04
75%,2.210000e+02,5.499700e+04
max,2.910000e+02,7.624700e+04


In [97]:
app_name_df = pd.DataFrame(df['app_name'].unique(), columns= ['App'])

pd.set_option('display.max_rows', None)

app_name_df

,App
0,Minesweeper Classic (Mines)
1,Gmail
2,Google
3,Instagram
4,Google Chrome
5,Clock
6,Maps
7,YouTube
8,Facebook
9,Messages


In [101]:
event_type_df = pd.DataFrame(df['event_type'].unique(), columns = ['Event Type'])

event_type_df

,Event Type
0,Opened
1,Closed
2,User Interaction
3,Broken


### Insights
1. The dataset has a size of 140MB and has 3.66M rows and has 5 columns.
2. The columns are:
    1. **User ID**. 292 unique users were recorded, each one had at least one session (261 sessions per user on average).
    2. **Session ID**. 72647 sessions were recorded, everyone accounting for multiple events (50 events per session on average).
    3. **timestamp**. Timestamp column stating the time at which a certain event was carried out.
    4. **App Name**. 86 different apps were recorded.
    5. **Event Type**. Event occured at that given time by that user in that session with that app. There are 4 of them (Opened, Closed, User Interaction and Broken).

### Step 4
Calculate new metrics:
1. **Event duration**. Calculate difference between consecutive events.
2. **Next event**. Highlight next event for future calculations and insights on user behaviour.
3. **Last event**. Highlight if an event is the last of the session or of the user.
4. **Part of day**. Calculate which part the day the timestamp refers to.

In [104]:
# Assuming df is your DataFrame after loading the dataset
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Calculate time difference between current and next event
df['next_timestamp'] = df.groupby('user_id')['timestamp'].shift(-1)
df['event_duration'] = (df['next_timestamp'] - df['timestamp']).dt.total_seconds()
df.drop(columns="next_timestamp", inplace=True)

# Create a column with the next event
df['next_event'] = df.groupby('user_id')['event_type'].shift(-1)

# Reset to default pandas display option
pd.reset_option('display.max_rows')

In [105]:
# Create two default columns "N"
df["is_session_last_event"] = "N"
df["is_user_last_event"] = "N"

# Create two masks to identify if it is user or session last event
mask_session = df["session_id"] != df["session_id"].shift(-1)
mask_user = df["user_id"] != df["user_id"].shift(-1)

# Write "Y" in the default "N" columns with the masks
df.loc[mask_session, "is_session_last_event"]  = "Y"
df.loc[mask_user, "is_user_last_event"]  = "Y"

In [106]:
# Add column to identify which part of the day the timestamp refers to
def get_part_of_day(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

df['part_of_day'] = df['timestamp'].dt.hour.apply(get_part_of_day)

df

,user_id,session_id,timestamp,app_name,event_type,event_duration,next_event,is_session_last_event,is_user_last_event,part_of_day
0,0,1,2018-01-16 06:01:05,Minesweeper Classic (Mines),Opened,0.0,Closed,N,N,Morning
1,0,1,2018-01-16 06:01:05,Minesweeper Classic (Mines),Closed,2.0,Opened,N,N,Morning
2,0,1,2018-01-16 06:01:07,Minesweeper Classic (Mines),Opened,0.0,Closed,N,N,Morning
3,0,1,2018-01-16 06:01:07,Minesweeper Classic (Mines),Closed,1.0,Opened,N,N,Morning
4,0,1,2018-01-16 06:01:08,Minesweeper Classic (Mines),Opened,0.0,Closed,N,N,Morning
...,...,...,...,...,...,...,...,...,...,...
3658584,291,76247,2018-04-06 14:35:15,Facebook,Closed,0.0,Opened,N,N,Afternoon
3658585,291,76247,2018-04-06 14:35:15,Facebook,Opened,22.0,Closed,N,N,Afternoon
3658586,291,76247,2018-04-06 14:35:37,Facebook,Closed,0.0,Opened,N,N,Afternoon
3658587,291,76247,2018-04-06 14:35:37,Facebook,Opened,15.0,User Interaction,N,N,Afternoon


### Step 5
In order to allow conversion analysis, a new column "is_converted" was added to identify if actually from a "user_interaction" event stemmed a conversion event (e.g., buying a product, subscribing to a service...).

In [77]:
# Create a new column "is converted" to simulate if the user interaction is converted
target_event_type = 'User Interaction'  # The event_type to check
probability = 0.2  # Probability of getting "Y" for the target_event_type

# Apply a function to create a new column based on conditions
df['is_converted'] = df['event_type'].apply(lambda x: np.random.choice(['Y', 'N'], p=[probability, 1-probability]) if x == target_event_type else 'N')

df

,user_id,session_id,timestamp,app_name,event_type,event_duration,next_event,is_session_last_event,is_user_last_event,part_of_day,is_converted
0,0,1,2018-01-16 06:01:05,Minesweeper Classic (Mines),Opened,0.0,Closed,N,N,Morning,N
1,0,1,2018-01-16 06:01:05,Minesweeper Classic (Mines),Closed,2.0,Opened,N,N,Morning,N
2,0,1,2018-01-16 06:01:07,Minesweeper Classic (Mines),Opened,0.0,Closed,N,N,Morning,N
3,0,1,2018-01-16 06:01:07,Minesweeper Classic (Mines),Closed,1.0,Opened,N,N,Morning,N
4,0,1,2018-01-16 06:01:08,Minesweeper Classic (Mines),Opened,0.0,Closed,N,N,Morning,N
...,...,...,...,...,...,...,...,...,...,...,...
3658583,291,76247,2018-04-06 14:34:34,Facebook,Opened,41.0,Closed,N,N,Afternoon,N
3658584,291,76247,2018-04-06 14:35:15,Facebook,Closed,0.0,Opened,N,N,Afternoon,N
3658585,291,76247,2018-04-06 14:35:15,Facebook,Opened,22.0,Closed,N,N,Afternoon,N
3658586,291,76247,2018-04-06 14:35:37,Facebook,Closed,0.0,Opened,N,N,Afternoon,N


### Step 6
Upload the new prepared file to **Google Cloud Storage**. 

In [114]:
def dataframe_to_gcs(bucket_name, df, destination_blob_name):
    # Create a GCS client
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    
    # Convert the DataFrame to a CSV in memory.
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)

    # Move to the beginning of the StringIO object.
    csv_buffer.seek(0)
    
    # Upload the in-memory CSV to GCS
    blob.upload_from_string(csv_buffer.getvalue(), content_type='text/csv')
    
    print(f"DataFrame uploaded to {destination_blob_name} in bucket {bucket_name}")

In [142]:
# Set your GCS bucket name and the GCS object name (path) where you want to upload the CSV.
bucket_name = 'event-app-usage'
destination_blob_name = 'app_data_prepared.csv'

# Perform the upload
dataframe_to_gcs(bucket_name, df, destination_blob_name)

SSLError: HTTPSConnectionPool(host='storage.googleapis.com', port=443): Max retries exceeded with url: /upload/storage/v1/b/event-app-usage/o?uploadType=resumable&upload_id=ABPtcPpAVGN7VVqL4n2SZ9jDpFAACX9g8sHo1NRHUX10VEyDmhD8DU8CBkMoxTuopc-AXqpI0C0jl0a881MXpIj4NOFf0rYDbojBVg1Cx7vfBais (Caused by SSLError(SSLWantWriteError(3, 'The operation did not complete (write) (_ssl.c:2406)')))

### Step 7
Using generative AI I created 3 side tables to have access to more data during the analysis:
1. **App Category**. The apps were categorized into 16 categories to allow aggregated analysis.
2. **User Demographic**. To every users demographic features were assigned.
3. **User App Feedbacks**. A rating from 1 to 5 was associated to every user-app pair.

In [134]:
# Display categorized app side table
bucket_name = 'event-app-usage'
blob_name = 'app_category.csv'
separator = ';'

pd.set_option('display.max_rows', None)

category_df = load_csv_to_dataframe(bucket_name, blob_name, separator)

category_df

,App Name,Category
0,Phone,Utilities
1,Clock,Utilities
2,Google Drive,Productivity
3,Pandora Music,Music & Audio
4,eBay,Shopping
5,Yahoo Mail,Communication
6,Calendar,Productivity
7,Google Play Music,Music & Audio
8,Google Photos,Photography
9,Discord,Social


In [135]:
# Display the categories
categories_df = pd.DataFrame(category_df['Category'].unique(), columns = ['Category'])

categories_df

,Category
0,Utilities
1,Productivity
2,Music & Audio
3,Shopping
4,Communication
5,Photography
6,Social
7,Games
8,Finance
9,News & Magazines


In [136]:
# Display feedbacks side table
bucket_name = 'event-app-usage'
blob_name = 'user_demographic.csv'
separator = ';'

pd.reset_option('display.max_rows')
pd.set_option('display.max_columns', None)

demographic_df = load_csv_to_dataframe(bucket_name, blob_name, separator)

demographic_df

,User ID,Age,Gender,Country,City Size,Employment Status,Education Level,Annual Income (USD),Marital Status,Interests,Number of Dependents,Home Ownership,Vehicle Ownership,Employment Sector,Hobbies,Dietary Preferences,Favorite Movie Genre,Travel Frequency,Physical Activity Level,Pet Ownership,Smoking Status,Alcohol Consumption
0,0,43,Male,Italy,Large,Employed,Bachelor's,"20,000-40,000",Married,Technology,2,Own,Own,Private,Outdoor activities,Vegan,Horror,Monthly,Moderately active,Cat,Occasional smoker,Social drinker
1,1,56,Female,Japan,Large,Employed,Master's,">100,000",Widowed,Technology,1,Rent,NaN,Private,Outdoor activities,Vegan,Fantasy,Quarterly,Moderately active,Cat,Occasional smoker,Social drinker
2,2,46,Female,Germany,Large,Employed,PhD,"80,001-100,000",Married,Travel,1,Rent,NaN,Unemployed,NaN,No preference,Comedy,Biannually,Moderately active,NaN,Non-smoker,Non-drinker
3,3,32,Female,Australia,Large,Employed,PhD,"20,000-40,000",Divorced,Music,2,Rent,Own,Private,Technology projects,Pescatarian,Horror,Monthly,Moderately active,Dog,Non-smoker,Social drinker
4,4,60,Female,Japan,Large,Employed,NaN,"40,001-60,000",Divorced,Technology,4,Own,NaN,Private,DIY,No preference,Fantasy,Never,Moderately active,Other,Non-smoker,Social drinker
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,288,42,Male,Australia,Medium,Student,NaN,"20,000-40,000",Single,Music,2,Other,NaN,Private,NaN,Vegetarian,Science Fiction,Monthly,Very active,NaN,Smoker,Social drinker
289,289,52,Male,Australia,Small,Retired,NaN,"60,001-80,000",Married,Gaming,2,Rent,Own,Public,Fitness,Vegetarian,Romance,Yearly,Sedentary,NaN,Occasional smoker,Regular drinker
290,290,42,Male,UK,Large,Unemployed,High School,"20,000-40,000",Widowed,Gaming,2,Other,NaN,Public,DIY,Meat-eater,Science Fiction,Quarterly,Moderately active,Other,Occasional smoker,Non-drinker
291,291,46,Other,USA,Large,Student,High School,"60,001-80,000",Divorced,Sports,2,Own,Own,Public,Volunteering,Pescatarian,Drama,Yearly,Moderately active,Dog,Smoker,Regular drinker


In [138]:
# Display feedbacks side table
bucket_name = 'event-app-usage'
blob_name = 'user_app_feedback.csv'
separator = ';'

feedback_df = load_csv_to_dataframe(bucket_name, blob_name, separator)

feedback_df

,user_id,app,user_feedback
0,0,Phone,4
1,0,Clock,1
2,0,Google Drive,3
3,0,Pandora Music,5
4,0,eBay,2
...,...,...,...
25399,291,Pixlr,5
25400,291,Flickr,1
25401,291,EntertaiNow,4
25402,291,DigiHUD Pro Speedometer,4


In [140]:
feedback_df.describe()

,user_id,user_feedback
count,25404.000000,25404.000000
mean,145.500000,3.002008
std,84.294304,1.417118
min,0.000000,1.000000
25%,72.750000,2.000000
50%,145.500000,3.000000
75%,218.250000,4.000000
max,291.000000,5.000000
